In [1]:
import pandas as pd
import numpy as np
import json
import stanza
from sklearn.metrics import classification_report

In [2]:
from utils import Utils

### Load data

In [3]:
!ls dataset

analyze-google-reviews.ipynb			google
analyze-google-topics.ipynb			google-booking-association
booking						google-booking-assoc.json
booking-translate.ipynb				hotel-association.ipynb
collocation-sentence.csv			split-data.ipynb
collocation-sentence-filtered.csv		trip-advisor
create-sent-dataset-for-training-vectors.ipynb	trip-advisor-translate.ipynb
data-analysis.ipynb				word_fixer.py
fix-incorrect-words.ipynb


In [29]:
!ls dataset/google-booking-association

google-booking-association.json  google-booking-hotels-train.json
google-booking-hotels-test.json


In [30]:
google_booking_hotels = Utils.load_json('dataset/google-booking-association/google-booking-association.json')

In [5]:
google_hotel_reviews = Utils.load_json('dataset/google/uk-topic-review-text.json')

In [6]:
booking_hotel_reviews = pd.read_csv('dataset/booking/booking-test.csv')

In [7]:
booking_hotel_reviews.head()

,Unnamed: 0,title,pos_text,neg_text,ratingValue,bestRating,hotel,rating
0,1786,"Хороше розташування , приємна ціна.","Хороше розташування , приємна ціна.","В душі багато павуків, грибок, обідрані дивани...",5.8,10.0,kiev-hotel-rock-wall.uk.html,3
1,1787,в цілому добре.,"Готель хороший, в хорошому місці,гарні номери,...",але ліжко двоспальне не передбачає зсунуті два...,7.9,10.0,kiev-hotel-rock-wall.uk.html,4
2,1788,Гарний будинок.,Гарний будинок. Вид з вікна зачаровує. Затишно...,В душі грибок...Ручка в дверях Коломана.,9.2,10.0,kiev-hotel-rock-wall.uk.html,5
3,1789,Персонал взагалі не відповідає ніяким критеріям.,Nan,Персонал взагалі не відповідає ніяким критерія...,4.5,10.0,kiev-hotel-rock-wall.uk.html,2
4,1790,Все\n,Все,Нічого,10.0,10.0,kiev-hotel-rock-wall.uk.html,5


### Stanza model

In [8]:
nlp = stanza.Pipeline('uk')

2020-06-13 10:21:54 INFO: Loading these models for language: uk (Ukrainian):
| Processor | Package |
-----------------------
| tokenize  | iu      |
| mwt       | iu      |
| pos       | iu      |
| lemma     | iu      |
| depparse  | iu      |

2020-06-13 10:21:54 INFO: Use device: gpu
2020-06-13 10:21:54 INFO: Loading: tokenize
2020-06-13 10:21:56 INFO: Loading: mwt
2020-06-13 10:21:56 INFO: Loading: pos
2020-06-13 10:21:57 INFO: Loading: lemma
2020-06-13 10:21:57 INFO: Loading: depparse
2020-06-13 10:21:58 INFO: Done loading processors!


### Collocation extractor

In [9]:
from collocations.noun_collocation_extractor import NounCollocationExtractor

In [10]:
collocation_extractor = NounCollocationExtractor(nlp)

In [11]:
text_examples = [
    "Чисто, охайно, чай/кава/вода безкоштовна, непогані сніданки і 5 хв до центра міста.",
    "дуже гарний готель, зі смачними сніданками",
    "на сніданок великий вибір страв",
    "Холодна та не свіжа їжа на сніданок",
    "Як на мене ідеальне місце для сніданку.",
    "В готелі є хороший вибір їжі на сніданок.",
    " Їжа смачна, обслуговування на високому рівні.",
    "Дуже приємний сніданок, сервіс найвищого рівня.",
    " В номері було так жарко що довелось відкривати вікна))) сніданки просто супер.",
    
    "Персонал дуже привітний, обслуговування банкету було на вищому рівні",
    "Хороший готель у центральній частині Львова. Чисті та затишні номери, хороші та смачні сніданки. Привітний персонал. Номери достатні по розміру з усім необхідним. Рекомендую для відпочинку."
    
]

In [12]:
collocation_examples = collocation_extractor.extract_for_texts(text_examples)

In [13]:
[Utils.generate_text_from_collocation(collocation_example) for collocation_example in collocation_examples]

['дуже гарний готель',
 'смачними сніданками',
 'на сніданок великий вибір страв',
 'холодна та не свіжа їжа на сніданок',
 'на мене ідеальне місце для сніданку',
 'є хороший вибір їжі на сніданок',
 'їжа смачна',
 'високому рівні',
 'дуже приємний сніданок',
 'відкривати вікна',
 'персонал дуже привітний',
 'обслуговування банкету було',
 'вищому рівні',
 'хороший готель',
 'центральній частині львова',
 'чисті та затишні номери',
 'хороші та смачні сніданки',
 'привітний персонал',
 'номери достатні']

In [14]:
text_example = "В готелі є хороший вибір їжі на сніданок."

In [15]:
doc = nlp(text_example)
doc.sentences[0].print_dependencies()

('В', '2', 'case')
('готелі', '3', 'obl')
('є', '0', 'root')
('хороший', '5', 'amod')
('вибір', '3', 'nsubj')
('їжі', '5', 'nmod')
('на', '8', 'case')
('сніданок', '5', 'nmod')
('.', '3', 'punct')


### FastText model

In [16]:
import fasttext

In [17]:
ft = fasttext.load_model('cc.uk.300.bin')

### Topic classifier 

In [18]:
from topic_classifier import TopicClassifier

In [19]:
topic_classifier_model_file = "models/topic-classifier.sav"

In [20]:
topic_classifier = TopicClassifier(topic_classifier_model_file, ft)

In [21]:
topic_classifier.predict(collocation_examples)[0]

{'Property': [('дуже гарний готель', 1.0),
  ('хороший готель', 1.0),
  ('чисті та затишні номери', 1.0),
  ('номери достатні', 1.0)],
 'Breakfast': [('смачними сніданками', 0.95),
  ('на сніданок великий вибір страв', 0.75),
  ('дуже приємний сніданок', 1.0),
  ('хороші та смачні сніданки', 1.0)],
 'Food and Beverage': [('холодна та не свіжа їжа на сніданок', 1.0),
  ('є хороший вибір їжі на сніданок', 0.95),
  ('їжа смачна', 1.0)],
 'Location': [('на мене ідеальне місце для сніданку', 1.0)],
 'Service': [('високому рівні', 1.0),
  ('персонал дуже привітний', 1.0),
  ('обслуговування банкету було', 0.65),
  ('вищому рівні', 1.0),
  ('привітний персонал', 1.0)],
 'Room amenities': [('відкривати вікна', 1.0)],
 'Atmosphere': [('центральній частині львова', 0.85)]}

### Collocation sentiment classifier

In [22]:
from sentiment_classifier import SentimentClassifier

In [23]:
collocation_sent_classifier = SentimentClassifier('models/sentiment-collocation.json', 
                                                  'models/sent-collocation-word-index.json')

In [24]:
collocation_sent_classifier.predict_sentiments_for_collocations(collocation_examples)

[0.99996793 0.9999827  0.9996362  0.9908552  0.9974269  0.99897385
 0.99996305 1.         1.         0.9999242  1.         0.99992096
 0.99999976 0.9999999  0.9999671  0.9999999  0.9999999  1.
 0.99984014]


['pos',
 'pos',
 'pos',
 'neg',
 'pos',
 'pos',
 'pos',
 'neut',
 'pos',
 'neut',
 'pos',
 'neut',
 'pos',
 'pos',
 'neut',
 'pos',
 'pos',
 'pos',
 'pos']

### Test full pipeline for several hotels

#### Test on google reviews

In [32]:
%%time
google_topic_sent_reports = Utils.calc_classification_report_for_each_topic(hotels = list(google_booking_hotels.keys()), 
                                                                      hotel_reviews=google_hotel_reviews, 
                                                                     collocation_extractor = collocation_extractor,
                                                                     topic_classifier=topic_classifier, 
                                                                     collocation_sent_classifier=collocation_sent_classifier)

[1.         1.         0.9993024  0.999998   0.99999917 0.9999304
 0.99999917 0.9999304  1.         0.9999943  0.9997899  0.9986248
 0.9999999  1.         1.         1.         0.9999982  0.99975735
 0.7155596  1.         1.         0.9999999  0.9952427  1.
 0.99999976 0.99999654 1.         1.         0.9999999  1.
 0.9999248  1.         0.965474   0.9990075  0.99999964 1.
 0.9998895  0.9990802  0.9999995  1.        ]
[0.99999917 0.99999917 0.99999917 0.9999615  0.99999917 0.99999917
 0.9999993  0.99999917 0.99999917 0.9999615  0.80944955 0.9999995
 0.99999917 0.99999917 0.9347416  0.99999917 0.99999917 0.9999988
 0.99999917 0.99978715 0.9999995  0.99999917 1.         0.99999917
 0.9999988  1.         0.9999615  0.9999999  1.         0.99999917
 0.99999917 0.99999917 0.97003156 1.         0.99999607 0.9999999
 0.9998727  0.9999988  0.99999917 0.98492014 1.         0.99999917
 1.        ]
[0.5702129  0.99998295 0.3974136  1.         0.88949865 1.
 0.3974136  1.         1.         0.9999

[0.99999845 0.9998006  0.7826216 ]
[0.9999993 0.9990736 0.9999981 0.999997  0.9991155 1.       ]
[0.99999976]
[0.9999988]
[1.         1.         0.99327636 0.99999654 1.         0.9999987
 0.9998574  0.9999999  0.9999999  0.9768644  1.         0.99988115
 1.         0.9999995  0.9999999  0.99999785 0.9999988  1.
 0.99999964 0.9999994  0.9999999  0.9999956  0.999997   0.9999918
 0.9985973  1.         0.9999949  0.999998   0.9936187  0.9999875
 0.99999106 1.         0.9999994  1.         0.9999335  0.9999825
 0.9703214  0.99964786 0.92889464 0.9999999  1.         1.
 0.9987557  0.9996375  0.99999964 0.9979163  0.9915507  0.99983335
 0.99997354 0.9999373  0.99998474 0.9997782  0.9999294  0.9367596
 0.9999999  1.         0.99999785 0.9999994  1.         0.9979584
 1.         0.99956065 0.99999976 1.         0.9999999  1.
 1.         1.         1.         1.         0.99992466 0.9999999
 0.99945706 0.9999677  1.         0.9999956 ]
[0.99873155 1.         0.99999917 1.         0.8372435  0.9

/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


[1.         0.99999964 1.         0.9999994  0.99998546 1.
 1.         0.74276453 1.         0.99999845 0.8265645  1.
 0.9999999  1.         0.9999949  1.         0.9999999  0.74985605
 0.999998   0.99979454 1.         1.         0.99978346 0.9999927
 0.99931526 1.         0.9999956  0.99999976 1.         0.99961424
 0.99999976 0.99999905 0.9999157  0.9999882  0.9925338  0.99997234
 0.9999995  1.         0.99999976 0.9996737  0.9999999  0.9993228
 0.99999297 0.99999976 1.         1.         0.9999999  0.9986852
 1.         0.99999774 1.         1.         1.         0.99992704
 0.99690753 0.9999994  0.9995882  0.99999285 0.5412194  0.6157758
 0.99999833 1.         1.         0.99999976 0.9999999 ]
[1.         0.9998807  0.99999917 1.        ]
[0.9999993  0.95650214 1.         0.99999976 1.         1.
 0.9999999  0.9997614  1.         0.9999993  0.99997306 1.
 1.         0.999671   0.99999905 0.9999907  0.9999993  1.
 0.9999981  0.9999968  0.9998337  1.         1.         0.99999475
 0.

[1.         0.999997   1.         0.9999968  0.9997936  0.9999993
 1.         1.         1.         1.         0.99708337 1.
 1.         0.88949865 0.9999994  1.         0.99999976 1.
 0.88949865 0.9999465  1.         0.9999993  0.9984957  1.
 1.         1.         1.         1.         1.         0.9999994
 0.99999976 0.99995995 0.9989504  1.         0.99999976 0.9998461
 0.9999993  1.         1.         1.         0.9189726  0.99890685
 0.78037524 0.9999995  0.9998915  0.999749   0.99999464 1.
 1.         1.         0.9999969  0.6316101  0.3974136  1.
 1.         1.         0.9984915  1.        ]
[1.         0.99999976 1.         0.9973091  0.9999938  0.9999864
 0.9414683  0.99975115 1.         0.99999845 0.99999976 1.
 1.         0.9999988  0.9999757  0.9999658  1.         1.
 0.9999999 ]
[0.9999999  0.53058165 0.99999917 0.9999615  0.99998474 0.9999615
 0.99999917 1.         0.99999917 0.99999917 1.         0.99999917
 1.         0.9999988  0.9949063  0.99999917 0.99999917 0.998891

...

[0.9998661  0.99835014 0.9999937 ]
[1.]
[0.99999964]
[0.9999939]
[0.92187816]
[0.999938  0.9999993]
[0.99999964 1.         0.99999285 1.         0.99980015 1.
 0.99997926 0.9999999 ]
[0.995475   0.9996722  0.9999999  0.9955745  0.99999714 0.99999976]
[0.9999981  1.         0.99989927 1.        ]
[1.         0.7773375  0.99870956]
[0.9999993]
[0.93154246 0.9738748 ]
[1.         0.9999976  0.9999999  0.98892874 0.9999995  0.9995888
 0.99997354 0.9999999  0.9999994  0.99997354 0.9999759  0.9997824
 0.51050436 0.99997354 0.99999964 1.         0.99997354]
[1.         1.         1.         0.82595795]
[1.         0.9999995  0.9999995  1.         0.99999964 0.7689074
 0.99999595 1.        ]
[1.        0.9999999 0.9999863 1.       ]
[0.9970216  0.99999905]
[0.99973017]
[0.999925]
[0.99999976 0.9993223  0.99999654 1.         1.         0.9999999 ]
[0.9999962]
[0.99997485 0.9908014  0.9999304  0.9999726  1.        ]
[0.9999993 0.9998779 0.9999999 0.9999999 1.       ]
[0.99973816]
[1.]
[0.9999474

[0.9867975]
[0.96885264]
[0.99970055]
[0.99996936 1.         0.99999905 1.         1.        ]
[0.99997354]
[1.]
[1.]
[0.9999999]
[0.99997723 0.9998325  0.99999976 0.99913836 1.         0.9994734
 0.99996066 0.9997173 ]
[0.5904504 0.9999999]
[0.99998426]
[0.9998579  0.69005543 0.9998242 ]
[1.]
[0.99999976]
[1.]
[0.99994147 1.         0.9999999  0.9999994  0.71829337 0.9999982
 1.         0.99983335 1.         1.         1.         0.9999999
 0.9996232  0.63784593 1.         0.9993223  0.99999964 1.        ]
[0.60437196 0.9998053  0.9999131  0.99315274 0.999688  ]
[0.9979887 1.        1.        1.        1.       ]
[1.]
[0.9999478]
[1.]
[1.         0.9999682  0.9942316  0.9999995  1.         1.
 1.         0.99999964]
[0.99997354 0.99999857 0.9999995  0.9999999  1.         0.9999956
 0.99663144 1.         0.9999995  0.9999969  0.8486999  1.
 0.98550165 0.99995244 0.99999547 0.9998909 ]
[0.9999795]
[0.99999726 0.9999969  1.         1.         0.5197665  0.91887045
 1.         1.         

[1.         0.9999914  0.9997379  0.99970824 0.9999304 ]
[1.         0.99999964 0.9999969  1.         1.         0.9790901 ]
[1.]
[0.99999213 0.9999809 ]
[0.99999774 0.8275521  0.99999964]
[0.99999595 0.9993315  0.9999088 ]
[0.99999845]
[1.         0.998689   0.9999994  0.9997875  0.99984264 1.        ]
[0.99993086 0.9999895  0.9995511 ]
[0.99991465 0.99999106]
[0.9993672  0.99998236]
[0.9996511  1.         0.99206924]
[0.9999937 0.9999647]
[0.9997379  0.99941325 0.99999404 1.         1.        ]
[0.99998283]
[0.99999857 0.99999905]
[1.]
[0.9998325 0.9999999 0.9998325]
[0.99849355 0.99999964 1.        ]
[1.         0.99795866 0.9999981  0.999129   0.99999785 0.99991155
 0.99999    1.         0.99999964 0.999998   0.9999924  1.        ]
[0.5501158 0.9999995 0.9999999 0.8196577 1.        1.        1.
 1.       ]
[0.999998  0.9999863]
[0.9999945 1.       ]
[1.         0.89007425]
[0.9999999]
[0.9999999 1.       ]
[0.9999616  0.99970645 1.        ]
[0.99999714]
[0.99991584 0.99999964 0.999

[1.         1.         1.         1.         1.         1.
 1.         1.         1.         0.9999999  0.99997354 0.9993113
 0.9979876  1.        ]
[0.9999999 0.9999639]
[0.99999976 1.         1.         0.99960333 0.9996375  0.99999976
 0.99999785 0.99999774 0.99999857 0.9979961  0.99998677 0.9999969
 0.9999999  0.9999999  0.99998724 0.9993223  1.         0.9999999
 0.99999726 1.         0.99999976 1.        ]
[0.9999999 1.       ]
[1.         1.         0.99999666 1.         0.999881   1.
 1.         1.         0.9998078  1.        ]
[0.9998907]
[0.99899]
[0.8771572]
[1.        0.9999734 0.9750329]
[0.9999429]
[0.99609524]
[1.]
[1.]
[0.9956398]
[0.9999982  0.9999877  1.         0.99999785 1.         0.9999838
 0.99999654 1.         0.999966   0.85464    0.99999654 0.9999969
 1.         0.99998724 0.9999864  0.9999958  0.9999999  0.99999976
 1.         0.9999958  0.99999976 1.         0.9999999  1.
 1.         1.         1.         1.         1.         1.
 0.99999976 0.9999932  0.99

/home/dbabenko/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/dbabenko/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
for topic in google_topic_sent_reports:
    print("TOPIC: ", topic)
    print(v)

TOPIC:  Location
              precision    recall  f1-score   support

         neg       0.25      0.57      0.35         7
        neut       0.00      0.00      0.00        43
         pos       0.88      0.98      0.93       311

    accuracy                           0.85       361
   macro avg       0.38      0.52      0.42       361
weighted avg       0.76      0.85      0.81       361

TOPIC:  Kitchen
              precision    recall  f1-score   support

         neg       0.23      0.55      0.32        49
        neut       0.00      0.00      0.00        74
         pos       0.56      0.57      0.56       121

    accuracy                           0.39       244
   macro avg       0.26      0.37      0.29       244
weighted avg       0.32      0.39      0.34       244

TOPIC:  Service
              precision    recall  f1-score   support

         neg       0.30      0.75      0.43        12
        neut       0.00      0.00      0.00        22
         pos       0.94   

In [34]:
google_topic_sent_reports[topic]

'              precision    recall  f1-score   support\n\n         neg       0.19      1.00      0.32        10\n        neut       0.00      0.00      0.00        42\n         pos       0.22      0.22      0.22        18\n\n    accuracy                           0.20        70\n   macro avg       0.14      0.41      0.18        70\nweighted avg       0.08      0.20      0.10        70\n'

# Demo result

#### select the same hotel for google and booking reviews

In [84]:
random_idx = np.random.choice(range(len(google_booking_hotels)))
selected_goole_hotel = list(google_booking_hotels.keys())[random_idx]
assert selected_goole_hotel in google_hotel_reviews
selected_goole_hotel

'Міні-готель SkyHome'

## Analyze from google reviews

In [85]:
selected_goole_hotel

'Міні-готель SkyHome'

In [86]:
%%time
hotel_collocations = collocation_extractor.extract_for_texts(google_hotel_reviews[selected_goole_hotel]['texts'])

CPU times: user 3.68 s, sys: 15.7 ms, total: 3.7 s
Wall time: 3.7 s


In [87]:
visible_collocations = [Utils.generate_text_from_collocation(collocation) for collocation in hotel_collocations]
visible_collocations

['відмінний готель',
 'сучасний ремонт',
 'загальний холодильник і бойлер',
 'дружелюбний персонал',
 'чудовий нічний вид з окна',
 'осталісь задоволені',
 'увечері шикарний вид з вікна',
 'маленькі кімнати з видом на балкон',
 'чудовий номер та умови проживання',
 'також окремої уваги заслуговує',
 'відмінне ставлення персоналу',
 'щирий прийом',
 'хороший відпочинок',
 'чудове місце для ночівлі',
 'інтимна обстановка',
 'хороший готель на околиці києва',
 'приємний адміністратор',
 'було більше посуди',
 'домашня атмосфера',
 'в стінах електричний гул',
 'персонал відкриває',
 'короче сервісом і не пахло',
 'відмінний сервіс',
 'привітний персонал',
 'демократичні ціни',
 'хороший персонал',
 'непоганий готельчик',
 'відмінний сервіс',
 'хороший міні',
 'міні готель',
 'прийнятними цінами',
 'ліжко стукала',
 'гарне місце для зупинки']

#### Classify topic of each collocation

In [88]:
visible_topic_collocations, topic_collocations = topic_classifier.predict(hotel_collocations)
visible_topic_collocations

{'Property': [('відмінний готель', 1.0),
  ('сучасний ремонт', 1.0),
  ('хороший відпочинок', 0.3),
  ('в стінах електричний гул', 1.0),
  ('демократичні ціни', 1.0),
  ('непоганий готельчик', 1.0),
  ('хороший міні', 0.7),
  ('міні готель', 1.0),
  ('прийнятними цінами', 1.0)],
 'Kitchen': [('загальний холодильник і бойлер', 1.0),
  ('було більше посуди', 0.7)],
 'Service': [('дружелюбний персонал', 1.0),
  ('осталісь задоволені', 0.75),
  ('чудовий номер та умови проживання', 0.5),
  ('також окремої уваги заслуговує', 0.9),
  ('відмінне ставлення персоналу', 0.85),
  ('щирий прийом', 0.9),
  ('приємний адміністратор', 1.0),
  ('персонал відкриває', 1.0),
  ('короче сервісом і не пахло', 1.0),
  ('відмінний сервіс', 1.0),
  ('привітний персонал', 1.0),
  ('хороший персонал', 1.0),
  ('відмінний сервіс', 1.0)],
 'Room amenities': [('чудовий нічний вид з окна', 1.0),
  ('увечері шикарний вид з вікна', 1.0),
  ('маленькі кімнати з видом на балкон', 0.9)],
 'Location': [('чудове місце для

In [89]:
 topic_collocations_sent, topic_summary = Utils.define_sentiments_and_summary(topic_collocations,
                                                                        visible_topic_collocations,
                                                                        collocation_sent_classifier, 
                                                                        topic_classifier,
                                                                       2)

In [90]:
topic_collocations_sent

{'Property': [('відмінний готель', 1.0, 'pos'),
  ('сучасний ремонт', 1.0, 'pos'),
  ('хороший відпочинок', 0.3, 'pos'),
  ('в стінах електричний гул', 1.0, 'neut'),
  ('демократичні ціни', 1.0, 'pos'),
  ('непоганий готельчик', 1.0, 'pos'),
  ('хороший міні', 0.7, 'pos'),
  ('міні готель', 1.0, 'neut'),
  ('прийнятними цінами', 1.0, 'pos')],
 'Kitchen': [('загальний холодильник і бойлер', 1.0, 'neut'),
  ('було більше посуди', 0.7, 'neut')],
 'Service': [('дружелюбний персонал', 1.0, 'pos'),
  ('осталісь задоволені', 0.75, 'pos'),
  ('чудовий номер та умови проживання', 0.5, 'pos'),
  ('також окремої уваги заслуговує', 0.9, 'neut'),
  ('відмінне ставлення персоналу', 0.85, 'pos'),
  ('щирий прийом', 0.9, 'pos'),
  ('приємний адміністратор', 1.0, 'pos'),
  ('персонал відкриває', 1.0, 'neut'),
  ('короче сервісом і не пахло', 1.0, 'neut'),
  ('відмінний сервіс', 1.0, 'pos'),
  ('привітний персонал', 1.0, 'pos'),
  ('хороший персонал', 1.0, 'pos'),
  ('відмінний сервіс', 1.0, 'pos')],
 '

In [91]:
Utils.print_hotel_summary(topic_summary, detailed=False)

	+ непоганий готельчик
	+ прийнятними цінами
	 загальний холодильник і бойлер
	 було більше посуди
	+ приємний адміністратор
	+ дружелюбний персонал
	 увечері шикарний вид з вікна
	 маленькі кімнати з видом на балкон
	+ чудове місце для ночівлі
	+ гарне місце для зупинки
	 інтимна обстановка
	 ліжко стукала


In [92]:
Utils.print_hotel_summary(topic_summary, detailed=True)


#######################################
TOPIC:  Property
SENTIMENT:  pos
	+ непоганий готельчик
	+ прийнятними цінами

#######################################
TOPIC:  Kitchen
SENTIMENT:  neut
	 загальний холодильник і бойлер
	 було більше посуди

#######################################
TOPIC:  Service
SENTIMENT:  pos
	+ приємний адміністратор
	+ дружелюбний персонал

#######################################
TOPIC:  Room amenities
SENTIMENT:  neut
	 увечері шикарний вид з вікна
	 маленькі кімнати з видом на балкон

#######################################
TOPIC:  Location
SENTIMENT:  pos
	+ чудове місце для ночівлі
	+ гарне місце для зупинки

#######################################
TOPIC:  Atmosphere
SENTIMENT:  neut
	 інтимна обстановка

#######################################
TOPIC:  Sleep
SENTIMENT:  neut
	 ліжко стукала


## Analyze from booking reviews

In [93]:
booking_hotel = google_booking_hotels[selected_goole_hotel]

In [94]:
booking_hotel_texts = Utils.get_booking_review_texts(booking_hotel_reviews, booking_hotel)

In [95]:
%%time
booking_collocations = collocation_extractor.extract_for_texts(booking_hotel_texts)

CPU times: user 39.7 s, sys: 201 ms, total: 39.9 s
Wall time: 39.9 s


In [96]:
booking_visible_collocations = [Utils.generate_text_from_collocation(collocation) for collocation in booking_collocations]
booking_visible_collocations

['персонал приємний',
 'персонал приємний',
 'зручним розташування готелю',
 'номер дуже маленький',
 'спальному ліжку',
 'ванній кімнаті',
 'душова кабіна була брудна',
 'не зрозуміла політика зміни',
 "сподобався інтер'єр",
 "сподобався інтер'єр",
 'має стоянки для авто',
 'не працювала лампа',
 'була напівтемрява',
 'хороший ремонт та меблі',
 'готель розташований',
 'останніх поверхах',
 'останній поверх ліфтом',
 'ціна доступна',
 'з спокоєм не було',
 'хороша адміністратор',
 'чарівна та приємна пані',
 'гарна панорама',
 'гарна панорама відкрилась',
 'поблизу є платна стоянка',
 'чути сусідів',
 'навіть коли ведуть звичайну розмову',
 'дуже тонкі стіни',
 'готель розміщений кухонний куток',
 'є можливість',
 'чай безкоштовний',
 'пральною машиною та холодильником',
 'постійно ведеться відеонагляд',
 'є кондиціонер',
 'ведеться будова',
 'постійний гуркіт техніки і стукіт',
 'рецепція знаходиться',
 'сусідньому під‘їзді',
 'другому під‘їзді',
 'готель розмістився',
 'фотографії н

In [97]:
booking_visible_topic_collocations, booking_topic_collocations = topic_classifier.predict(booking_collocations)
booking_visible_topic_collocations

{'Service': [('персонал приємний', 1.0),
  ('персонал приємний', 1.0),
  ('не зрозуміла політика зміни', 0.55),
  ('з спокоєм не було', 0.35),
  ('хороша адміністратор', 1.0),
  ('чарівна та приємна пані', 0.95),
  ('чути сусідів', 0.85),
  ('навіть коли ведуть звичайну розмову', 0.55),
  ('рецепція знаходиться', 1.0),
  ("обов'язковому порядку", 0.7),
  ('оплата тільки готівкою+ беруть завдаток', 0.6),
  ('ввічливий персонал', 1.0),
  ('нормальному розумінні', 0.95),
  ('привітний персонал', 1.0),
  ('прекрасне відношення персоналу', 0.95),
  ('прекрасне відношення персоналу', 0.95),
  ('приємно здивувало співвідношення', 0.55),
  ('адміністратор зажадала', 1.0),
  ('сплачуючи комісію банку', 0.7),
  ('персонал швидко відреагував', 1.0),
  ('персонал швидко відреагував', 1.0),
  ('безсовісний адміністратори', 1.0),
  ('адміністратор не прийшла', 1.0),
  ('адміністратор погодилася', 1.0),
  ('адміністратор і каже', 1.0),
  ('сплатити заставу', 0.95),
  ('враження неоднозначні', 0.6),
 

In [98]:
booking_topic_collocations_sent, booking_topic_summary = Utils.define_sentiments_and_summary(booking_topic_collocations,
                                                                                       booking_visible_topic_collocations,
                                                                                       collocation_sent_classifier, 
                                                                                        topic_classifier,
                                                                                       2)

In [99]:
#booking_topic_collocations_sent

In [101]:
Utils.print_hotel_summary(booking_topic_summary, detailed=False)

	+ дуже привітно і чудовий адміністратор
	+ дуже привітно і чудовий адміністратор
	 поставили додаткове місце
	 місце зовсім не в центрі
	 пройти ряд коридорів від ліфта
	 саме там розташовані номери
	 дуже погана звукоізоляція
	 дуже погана звукоізоляція
	 вранці) гарячої води просто не застав
	 дуже погано працювала каналізація
	 поблизу є платна стоянка
	 має стоянки для авто
	 открывается вид на місто
	 тому вид з вікна на кам
	 мансардним поверсі
	 останніх поверхах
	 фото відповідають дійсності
	 фото відповідають дійсності
	 готель розміщений кухонний куток
	 так же холодильник
	 загальна кухня
	 фотографії не відповідають реальності
	 термінал не працював
	 загальній культурі населення
	 кухонними зоною
	 розрахувати ся
	 зробленим вибором на користь
	 немає можливості перекладу
	+ прибирання не повинна


### Generated text summary about hotel 

In [ ]:
#TODO:

In [83]:
#TODO: